In [1]:
from selenium import webdriver  
from selenium.webdriver.common.by import By
import time  
import urllib  
from slugify import slugify
import requests 
import random
import os
import re


# 下载方式

1. 需要找到论文列表页，例如对于cvpr，我们进入官网https://openaccess.thecvf.com/CVPR2023  （带图的在download中）

2. 发现有All parper，点击后进入，然后就进入一个全是论文的列表页了。这个列表页对应的网址就是我们要填的网址

3. 最后，选择主题和存放路径后就可以爬取了。

# 修改内容

1. 增加主题和关键字检测
2. 在获得所有论文的列表后，进行关键字筛选
3. 在最后保存文件的时候，要对文件名部分做修改，让文件名更容易读。


In [2]:
# CVPR 2023 DETECT
# conference_url = "https://openaccess.thecvf.com/CVPR2023?day=all" # the conference url to download papers from
# Edgedriver_path = r'./supports/msedgedriver.exe'.replace('\\','/') # the chromedriver.exe path
# # 主题
# subjcet  = "Detect" # 论文主题
# pattens = ["Detect", "DETR", "RCNN", "YOLO"]  #  要检测的关键字
# root = r'D:\Papers\CVPR2023_' + subjcet
# root.replace('\\','/')  # 设置本次下载的目录

In [3]:
conference_url = "https://openaccess.thecvf.com/ICCV2023?day=all" # the conference url to download papers from
Edgedriver_path = r'./supports/msedgedriver.exe'.replace('\\','/') # the chromedriver.exe path
# 主题
subjcet = "Detect"  # 论文主题
pattens = ["Detect", "DETR", "RCNN", "YOLO"]  #  要检测的关键字
root = r'D:\Papers\ICCV2023_' + subjcet
root.replace('\\','/')  # 设置本次下载的目录


root

'D:\\Papers\\ICCV2023_Detect'

In [4]:
os.makedirs(root, exist_ok=True)
print(root)
print(Edgedriver_path)
driver = webdriver.Edge()  
driver.get(conference_url)


D:\Papers\ICCV2023_Detect
./supports/msedgedriver.exe


In [5]:
# 获得所有论列表
pdfurllist =  []
pdfnamelist = []

# title_element_list = driver.find_elements_by_class_name('ptitle')
# url_element_list = driver.find_elements_by_partial_link_text('pdf')
title_element_list = driver.find_elements(by=By.CLASS_NAME, value='ptitle')
url_element_list = driver.find_elements(by=By.PARTIAL_LINK_TEXT, value='pdf')
for i, element in enumerate(url_element_list): 

    pdfnamelist.append(title_element_list[i].text)
    pdfurllist.append(url_element_list[i].get_attribute('href'))



In [6]:
print("All paper is:", len(pdfurllist))

All paper is: 2156


In [7]:
# 根据主题筛选论文
download_pdfurllist = []
download_pdfnamelist = []
# 筛选包含关键词的论文
for i in range(len(pdfurllist)): 
    download_flag = False  # 判断是否下载的变量
    for p in pattens:
        if re.search(p, pdfurllist[i]):  # 检索到关键词就设置True并跳出
            download_flag = True
            break
    # flag为 True 就下载
    if download_flag:
        download_pdfurllist.append(pdfurllist[i])
        download_pdfnamelist.append(pdfnamelist[i])
print("Total paper number in " + subjcet + " is: " + str(len(download_pdfurllist)))
        

Total paper number in Detect is: 171


In [8]:
# check the retrieved urls
print('The first 5 pdf urls:\n')
for i in range(5):
    print(download_pdfurllist[i])
print('\nThe last 5 pdf urls:\n')
for i in range(5):
    print(download_pdfurllist[-(i+1)])
print('=======================================================')


The first 5 pdf urls:

https://openaccess.thecvf.com/content/ICCV2023/papers/Zhang_A_Dynamic_Dual-Processing_Object_Detection_Framework_Inspired_by_the_Brains_ICCV_2023_paper.pdf
https://openaccess.thecvf.com/content/ICCV2023/papers/Cao_E2E-LOAD_End-to-End_Long-form_Online_Action_Detection_ICCV_2023_paper.pdf
https://openaccess.thecvf.com/content/ICCV2023/papers/Xu_TALL_Thumbnail_Layout_for_Deepfake_Video_Detection_ICCV_2023_paper.pdf
https://openaccess.thecvf.com/content/ICCV2023/papers/He_Bidirectional_Alignment_for_Domain_Adaptive_Detection_with_Transformers_ICCV_2023_paper.pdf
https://openaccess.thecvf.com/content/ICCV2023/papers/Zhao_Unified_Visual_Relationship_Detection_with_Vision_and_Language_Models_ICCV_2023_paper.pdf

The last 5 pdf urls:

https://openaccess.thecvf.com/content/ICCV2023/papers/Ye_Efficient_Transformer-based_3D_Object_Detection_with_Dynamic_Token_Halting_ICCV_2023_paper.pdf
https://openaccess.thecvf.com/content/ICCV2023/papers/Hornauer_Out-of-Distribution_Detec

In [9]:
# check the retrieved paper titles
print('The first 5 pdf titles:\n')
for i in range(5):
    print(download_pdfnamelist[i])
print('\nThe last 5 pdf titles:\n')
for i in range(5):
    print(download_pdfnamelist[-(i+1)])

The first 5 pdf titles:

A Dynamic Dual-Processing Object Detection Framework Inspired by the Brain's Recognition Mechanism
E2E-LOAD: End-to-End Long-form Online Action Detection
TALL: Thumbnail Layout for Deepfake Video Detection
Bidirectional Alignment for Domain Adaptive Detection with Transformers
Unified Visual Relationship Detection with Vision and Language Models

The last 5 pdf titles:

Efficient Transformer-based 3D Object Detection with Dynamic Token Halting
Out-of-Distribution Detection for Monocular Depth Estimation
CoTDet: Affordance Knowledge Prompting for Task Driven Object Detection
Inter-Realization Channels: Unsupervised Anomaly Detection Beyond One-Class Classification
Label-Efficient Online Continual Object Detection in Streaming Video


In [10]:
print('The number of papers is ', len(download_pdfnamelist))
assert len(download_pdfnamelist)==len(download_pdfurllist), 'The number of titles and the number of urls are not matched. \
                                            You might solve the problem by checking the HTML code in the \
                                            website yourself or you could ask the author by raising an issue.'



The number of papers is  171


In [11]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(download_pdfurllist):
    if url != None :      
        pdfname = slugify(download_pdfnamelist[i], lowercase=False, separator="_")  # 设置下划线为分隔符
        if os.path.isfile(root+'/'+pdfname+".pdf"):
            print('existed', i, '\t', download_pdfnamelist[i], '\t', download_pdfurllist[i])
        else:
            print(i, '\t', download_pdfnamelist[i], '\t', download_pdfurllist[i])
            data = requests.get(download_pdfurllist[i], timeout=80, headers=headers).content
            
            with open(os.path.join(root,pdfname+".pdf"), 'wb')  as f:
                f.write(data)  
            _ = time.sleep(random.uniform(4,5)) # for anti-anti-crawler purpose

Start downloading
existed 0 	 A Dynamic Dual-Processing Object Detection Framework Inspired by the Brain's Recognition Mechanism 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Zhang_A_Dynamic_Dual-Processing_Object_Detection_Framework_Inspired_by_the_Brains_ICCV_2023_paper.pdf
existed 1 	 E2E-LOAD: End-to-End Long-form Online Action Detection 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Cao_E2E-LOAD_End-to-End_Long-form_Online_Action_Detection_ICCV_2023_paper.pdf
existed 2 	 TALL: Thumbnail Layout for Deepfake Video Detection 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Xu_TALL_Thumbnail_Layout_for_Deepfake_Video_Detection_ICCV_2023_paper.pdf
existed 3 	 Bidirectional Alignment for Domain Adaptive Detection with Transformers 	 https://openaccess.thecvf.com/content/ICCV2023/papers/He_Bidirectional_Alignment_for_Domain_Adaptive_Detection_with_Transformers_ICCV_2023_paper.pdf
existed 4 	 Unified Visual Relationship Detection with Vision and Language Models 	

149 	 DistillBEV: Boosting Multi-Camera 3D Object Detection with Cross-Modal Knowledge Distillation 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Wang_DistillBEV_Boosting_Multi-Camera_3D_Object_Detection_with_Cross-Modal_Knowledge_Distillation_ICCV_2023_paper.pdf
150 	 Detecting Objects with Context-Likelihood Graphs and Graph Refinement 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Bhowmik_Detecting_Objects_with_Context-Likelihood_Graphs_and_Graph_Refinement_ICCV_2023_paper.pdf
151 	 Self-Supervised Object Detection from Egocentric Videos 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Akiva_Self-Supervised_Object_Detection_from_Egocentric_Videos_ICCV_2023_paper.pdf
152 	 NeSS-ST: Detecting Good and Stable Keypoints with a Neural Stability Score and the Shi-Tomasi detector 	 https://openaccess.thecvf.com/content/ICCV2023/papers/Pakulev_NeSS-ST_Detecting_Good_and_Stable_Keypoints_with_a_Neural_Stability_ICCV_2023_paper.pdf
153 	 Rethinking Pose Estimation in